# Projeção de ocupação de leitos enfermaria e UTI em estados com modelos SEIR e SEAPMDR

Originalmente, a ferramenta **SimulaCovid** utiliza um modelo epidemiológico SEIR (Suscetíveis, Expostos, Infectados e Removidos) para calcular a disseminação e evolução clínica do COVID-19. Esse modelo original era baseado na implementação desenvolvida por [Hill (2020)](https://github.com/alsnhll/SEIR_COVID19) e dividia a população em diferentes "compartimentos"
mutuamente excludentes de indivíduos:

- Suscetíveis ($S$): aqueles que não têm imunidade à doença, estes se tornam
expostos à doença a uma taxa $\\beta_i$ devido ao contato com indivíduos
infetados $I_i$;
- Expostos ($E$): aqueles que entram em contato com a doença e desenvolvem
ou não sintomas, mas ainda não transmitem infecção. Esses indivíduos
expostos progridem para o primeiro estágio de infecção ($I_1$), leve, a
uma taxa $\sigma$.
- Infectados ($I$): são aqueles que desenvolvem sintomas e transmitem a
doença a uma taxa $\beta_i$. Os casos de infecção são separados em
diferentes estágios de gravidade:
- Leve ($I_1$): aquele que não necessita de hospitalização, progride para
    o estado severo a uma taxa $p_1$, ou se recupera a uma taxa $\gamma_1$;
- Severo ($I_2$): aquele que necessita de hospitalização, progride para o
    estado crítico a uma taxa $p_1$, ou se recupera a uma taxa $\gamma_2$;
- Grave ($I_3$): aquele que além de hospitalização, necessita de
    tratamento intensivo com equipamento de ventiladores. Chegando ao caso
    grave, o indivíduo pode se recuperar a uma taxa $\gamma_3$ ou o quadro
    pode levar à morte com uma taxa $\mu$;
- Recuperados ($R$): aqueles que, após o curso da doença, se recuperam e
desenvolvem imunidade - não retornam ao estado suscetível.
- Mortos ($D$): indivíduos que morrem pelo agravamento da infecção.

Conforme a descrição acima, o conjunto de equações que determinavam a dinâmica do modelo era dado por:

$\frac{dS}{dt} = - (\beta_1 I_1 + \beta_2 I_2 + \beta_3 I_3) S$

$\frac{dE}{dt} = (\beta_1 I_1 + \beta_2 I_2 + \beta_3 I_3) S - \sigma E$

$\frac{dI_1}{dt} = \sigma E - (γ_1 + p_1) I_1$

$\frac{dI_2}{dt} = p_1 I_1 - (γ_2 + p_2) I_2$

$\frac{dI_3}{dt} = p_2 I_2 - (γ_3 + μ) I_3$

$\frac{dR}{dt} = γ_1 I_1 + γ_2 I_2 + γ_3 I_3$

$\frac{dD}{dt} = μ I_3$


### Atualizações do modelo

Esse modelo, no entanto, ignora algumas características importantes da transmissão da Covid-19, que foram se tornando mais claras com a evolução do conhecimento sobre a doença. 

Em especial, diversos trabalhos (por exemplo, [Tindale et al., 2020]; [Lee et
  al., 2020]; e [Casey et al., 2020]) têm destacado a existência de transmissão secundária por pacientes pré-sintomáticos (expostos, mas que ainda não desenvolveram sintomas) e assintomáticos (infectados que jamais desenvolverão sintomas). Do ponto de vista da saúde pública, a ausência desses grupos na modelagem epidemiológica pode levar a uma subestimativa a taxa de transmissão no início de um surto local, e a uma superestimativa no número de mortes ([Yang, Lombardi Jr. e Yang, 2020b]).

Para mitigar essa possibilidade, implementamos um modelo mais representativo da história natural da enfermidade. Assim como o modelo original, esse novo modelo foi baseado na implementação proposta por [Hill (2020)] - que, desde março de 2020, também foi alterada para comportar grupos de indivíduos assintomáticos e pré-sintomáticos capazes de transmitir a doença. Com esses novos compartimentos, a implementação se aproxima mais de um modelo SEAPMDR (Suscetíveis \[S], Expostos \[E], Assintomáticos \[A], Pré-sintomáticos \[P], Moderados [M], Severos \[D] e Recuperados \[R]), conforme proposto por [Yang, Lombardi Jr. e Yang (2020a)].

Com os novos compartimentos, o grupo de Expostos ($E$) original passa a ser dividido em um compartimento $E_0$, de indivíduos expostos com o agente infeccioso latente (não transmissível); e um compartimento $E_1$, composto por indivíduos pré-sintomáticos (e transmissíveis):

- Latentes ($E_0$): aqueles que entraram em contato com a doença mas ainda não desenvolveram a doença nem se tornaram transmissíveis. Progridem para o estágio $E_1$ a uma taxa $\sigma_0$.
- Pré-sintomáticos ($E_0$): aqueles que entraram em contato com a doença, ainda não desenvolveram a doença, mas já transmitem o agente infeccioso a uma taxa $\beta_E$. Progridem para o estágio $I_1$ ou $I_0$ *(ver abaixo)* a uma taxa $\sigma_1$.

Além dos casos pré-sintomáticos, foi considerado um novo subcompartimento de infectados: aqueles que, passado o período de incubação, jamais chegam a desenvolver sintomas - mas continuam sendo transmissíveis durante um certo período, antes de se recuperarem completamente. Esse subcompartimento foi denominado $I_0$, e comporta uma proporção $\phi$ dos indivíduos que entram no compartimento $I$ (Infectados):

- Assintomáticos ($I_0$): aqueles indivíduos que já superaram o período de incubação, mas não chegam a demonstrar sintomas. Ainda assim, transmitem a doença a uma taxa $\beta_0$. Se recuperam completamente (e deixam de ser transmissíveis) a uma taxa $\gamma_0$.

Com essas mudanças, as equações atualizadas do modelo passam a ser:


$\frac{dS}{dt} = - (\beta_E E_1 + \beta_1 I_1 + \beta_2 I_2 + \beta_3 I_3) S$

$\frac{dE_0}{dt} = (\beta_E E_1 + \beta_1 I_1 + \beta_2 I_2 + \beta_3 I_3) S - \sigma_0 E_0$

$\frac{dE_1}{dt} = \sigma_0 E_0 - \sigma_1 E_1$

$\frac{dI_0}{dt} = \sigma_1 E_1 \phi - \gamma_0 I_0$

$\frac{dI_1}{dt} = \sigma_1 E_1 (1- \phi) - (\gamma_1 + p_1) I_1$

$\frac{dI_2}{dt} = p_1 I_1 - (\gamma_2 + p_2) I_2$

$\frac{dI_3}{dt} = p_2 I_2 - (\gamma_3 + μ) I_3$

$\frac{dR}{dt} = \gamma_1 I_1 + \gamma_2 I_2 + \gamma_3 I_3$

$\frac{dD}{dt} = \mu I_3$

#### Outras alterações

Outra mudança realizada em relação à implementação original foi a utilização da taxa de transmissão efetiva estimada para o local para calcular a proporção de indivíduos suscetíveis ($S$) são expostos à doença (entram para o compartimento $E_0$) a cada intervalo de tempo. 

Anteriormente, esse cálculo era realizado com uma taxa padrão para a doença - o que ignora as trajetórias potencialmente muito diferentes da epidemia em cada local. Ao invés disso, são utilizadas as estimativas já geradas pelo FarolCovid para as taxas de transmissão locais, seguindo a fórmula (disponibilizada por Wicklin, 2020, entre outros):

$D = \ln(2) / R_t$

onde $D$ é o número de dias para que o número de casos dobre; e $R_t$ é a estimava da taxa de transmissão local.

[Casey et al., 2020]: https://doi.org/10.1101/2020.05.08.20094870
[Hill (2020)]: https://github.com/alsnhll/SEIR_COVID19
[Lee et al., 2020]: https://doi.org/10.1007/s12630-020-01729-x
[Tindale et al., 2020]: https://doi.org/10.7554/eLife.57149
[Yang, Lombardi Jr. e Yang (2020a)]: https://arxiv.org/abs/2004.05715
[Yang, Lombardi Jr. e Yang, 2020b]: https://doi.org/10.1101/2020.10.11.20210831


## Importa bibliotecas

In [7]:
from pathlib import Path

import pandas as pd
import requests
import yaml


# HACK: allows sibling directory imports in notebooks
# CREDIT: https://stackoverflow.com/a/60478241
import sys
sys.path.insert(0, '..') # add parent folder path where lib folder is
from simulacovid import prepare, seir, seapmdr, simulator

## Como rodar a simulação para os estados

### Carrega parâmetros fixos do arquivo de configuração

A ferramenta [**FarolCovid**](https://github.com/ImpulsoGov/farolcovid) disponibiliza um arquivo de configuração que inclui dados dos tempos de progressão da doença, preparados para os cálculos do simulador. 

Como a versão SEAPMDR exige alguns parâmetros inexistentes nas configurações originais, foi disponibilizado [no repositório](https://github.com/bcbernardo/impulsogov-desafio-simulacovid/blob/master/custom_configs.yaml) uma versão modificada com as configurações relevantes.

In [8]:
# Get from GitHub
#url = (
#    "https://raw.githubusercontent.com/bcbernardo/impulsogov-desafio-simulacovid/" +
#    "master/custom_configs.yaml"
#)

#config = yaml.load(requests.get(url).text, Loader=yaml.FullLoader)

# OR get local
with open(Path("..", "custom_configs.yaml"), "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

config["br"]["seir_parameters"]

{'asymptomatic_proportion': 0.3,
 'asymptomatic_duration': 6,
 'mild_duration': 6,
 'severe_duration': 6,
 'critical_duration': 8,
 'fatality_ratio': 0.02,
 'doubling_rate': 1.15,
 'incubation_period': 5,
 'presymptomatic_period': 2,
 'i0_percentage': 0.256,
 'i1_percentage': 0.599,
 'i2_percentage': 0.092,
 'i3_percentage': 0.019,
 'infected_health_care_proportion': 0.05,
 'hospitalized_by_age_perc': {'from_0_to_9': 2e-05,
  'from_10_to_19': 0.0004,
  'from_20_to_29': 0.011,
  'from_30_to_39': 0.034,
  'from_40_to_49': 0.043,
  'from_50_to_59': 0.082,
  'from_60_to_69': 0.118,
  'from_70_to_79': 0.166,
  'from_80_to_older': 0.184}}

### Carrega parâmetros de hospitalização e mortalidade por estado

**Conforme a implementação original**, passamos a utilizar parâmetros calculados por nível geográfico (neste caso estado) levando em consideração a distriuição etária da população. Abaixo segue um trecho de a metodologia com mais detalhes sobre esse cálculo:

> A proporção esperada de hospitalizações é dada pela estimativa de hospitalizações de cada faixa etária, de Verity, Robert, et al. (2020) num amplo estudo com base em 3.665 casos na China, ponderado pelo total da população em cada faixa etária (População residente em 2019 - CNES). Esse valor nos fornece o percentual total de hospitalizações esperadas na população (I2+I3/I). Para obter o percentual por intensidade do caso, mantivemos a razão entre severos (I2) e críticos (I3) constante: antes tínhamos 12.5% severos e 2.5% críticos do total de casos ativos (I) - ou seja, uma razão de 0.2 críticos para cada caso severo (I3/I2) -, que passa a ser de 83.3% severos (I2) e 16.7% críticos (I3) do total estimado de hospitalizados (I2+I3 / I).

Para a incorporação da categoria de assintomáticos, levou-se em conta uma proporção de 30% dos novos casos (assim como faz Hill (2020), citando Bi et al. 2020; Mizumoto et al. 2020; e Nishiura et al. 2020). O 70% restante dos casos corresponde aos indivíduos que desenvolvem a forma moderada da doença, podendo progredir para os próximos estágios. **Ao contrário da implementação original**, aqui calculamos explicitamente o percentual de assintomáticos da população - ao invés de deixar essa tarefa a cargo de uma subrotina dentro do código.

In [9]:
place_id = "state_num_id"

In [10]:
place_specific_params = pd.read_csv("http://datasource.coronacidades.org/br/states/parameters")
place_specific_params.set_index(place_id, inplace=True)

# recalculate severity percentages considering the asymptomatic fraction
asymptomatic_proportion = config["br"]["seir_parameters"]["asymptomatic_proportion"]
place_specific_params["i0_percentage"] = asymptomatic_proportion
place_specific_params[["i1_percentage", "i2_percentage", "i3_percentage"]] = (
    place_specific_params[["i1_percentage", "i2_percentage", "i3_percentage"]]
    .apply(lambda x: x * ( 1 - asymptomatic_proportion))
)

place_specific_params

,fatality_ratio,hospitalized_by_age_perc,i1_percentage,i2_percentage,i3_percentage,data_last_refreshed,i0_percentage
state_num_id,,,,,,,
11,0.004582,0.035272,0.675310,0.020433,0.004257,2021-01-30 01:09:09,0.3
12,0.003882,0.029194,0.679565,0.016912,0.003523,2021-01-30 01:09:09,0.3
13,0.003741,0.029431,0.679399,0.017049,0.003552,2021-01-30 01:09:09,0.3
14,0.003333,0.029069,0.679652,0.016840,0.003508,2021-01-30 01:09:09,0.3
15,0.004362,0.032281,0.677403,0.018701,0.003896,2021-01-30 01:09:09,0.3
16,0.003405,0.028235,0.680235,0.016357,0.003408,2021-01-30 01:09:09,0.3
17,0.005203,0.035553,0.675113,0.020596,0.004291,2021-01-30 01:09:09,0.3
21,0.005125,0.033957,0.676230,0.019672,0.004098,2021-01-30 01:09:09,0.3
22,0.006147,0.039047,0.672667,0.022620,0.004713,2021-01-30 01:09:09,0.3


### Obtém os dados epidemiológicos de entrada

#### Importa dados históricos dos estados

**Ao contrário da implementação original**, utilizamos uma [versão customizada](https://github.com/bcbernardo/coronacidades-datasource) da ferramenta CoronaCidades, com uma [extremidade adicional](https://github.com/bcbernardo/coronacidades-datasource/blob/master/src/loader/endpoints/get_states_farolcovid_history.py) para registrar o histórico de evolução dos dados de:

- Casos e óbitos acumulados até cada data desde os primeiros casos nos estados
- Taxa de notificação e casos ativos estimados, tomando os 15 dias anteriores à data de referência
- Taxa de contágio (Rt), também tomando os 15 dias anteriores

Como a geração desses dados pode levar algumas horas, uma versão consolidada dos dados é disponibilizada localmente no repositório, no caminho `data/br-states-farolcovid-history.csv`, e na cópia do repositório no GitHub.

In [11]:
df_farol = pd.read_csv("../data/br-states-farolcovid-history.csv")

# fix missing ids
state_num_ids = {  # from https://servicodados.ibge.gov.br/api/v1/localidades/estados/
  "RO": 11,
  "AC": 12,
  "AM": 13,
  "RR": 14,
  "PA": 15,
  "AP": 16,
  "TO": 17,
  "MA": 21,
  "PI": 22,
  "CE": 23,
  "RN": 24,
  "PB": 25,
  "PE": 26,
  "AL": 27,
  "SE": 28,
  "BA": 29,
  "MG": 31,
  "ES": 32,
  "RJ": 33,
  "SP": 35,
  "PR": 41,
  "SC": 42,
  "RS": 43,
  "MS": 50,
  "MT": 51,
  "GO": 52,
  "DF": 53,
}
df_farol["state_num_id"] = df_farol["state_id"].apply(lambda UF: state_num_ids[UF])

# erase rows with missing data
df_farol = (
    df_farol
    .dropna()
    .reset_index()
)

df_farol["last_updated_cases"] = pd.to_datetime(df_farol["last_updated_cases"])

df_farol[["state_num_id", "state_id", "last_updated_cases", "confirmed_cases", "deaths", 
          "notification_rate", "active_cases", "rt_most_likely"]]

,state_num_id,state_id,last_updated_cases,confirmed_cases,deaths,notification_rate,active_cases,rt_most_likely
0,35,SP,2020-03-28,1406.0,84.0,0.014600,91849.0,2.84
1,35,SP,2020-03-29,1451.0,98.0,0.013834,100185.0,2.78
2,35,SP,2020-03-30,1517.0,113.0,0.013900,98201.0,2.75
3,35,SP,2020-03-31,2339.0,136.0,0.021356,101843.0,2.61
4,35,SP,2020-04-01,2981.0,164.0,0.027297,100416.0,2.65
...,...,...,...,...,...,...,...,...
7616,43,RS,2021-01-19,512343.0,10051.0,0.515102,106499.0,0.96
7617,50,MS,2021-01-18,152026.0,2705.0,0.343943,45816.0,1.03
7618,51,MT,2021-01-19,202125.0,4846.0,0.211953,46388.0,1.13
7619,52,GO,2021-01-19,332391.0,7107.0,0.251854,38367.0,1.04


### Roda a simulação para cada registro histórico

**Ao contrário da implementação original**, preparamos e rodamos uma simulação para cada modelo (SEIR original e SEAPMDR) e para cada registro epidemiológico diário de cada UF desde o início dos casos. Isso permite avaliar o quanto cada um dos modelos acertou historicamente, comparando seu desempenho.

In [12]:
# create an empty DataFrame to receive predictions
df_predictions = pd.DataFrame()

for _, row in df_farol.iterrows():
    
    # prepare simulation parameters (S, E0, E1, I0, I1, I2, I3, R, D, Rt, N)
    params = prepare.prepare_simulation(row, place_id, config, place_specific_params)
    
    # run one simulation for each model available
    for model in ["SEAPMDR", "SEIR"]:
        
        # run simulation and get (best and worst) scenarios
        dfs_scenarios = simulator.run_simulation(params, config, model=model)
        
        # unpack scenarios
        for scenario in ["worst", "best"]:
            df = dfs_scenarios[scenario]
            df["scenario"] = scenario
            
            df["date_prediction"] = row["last_updated_cases"]
            df["state_num_id"] = row["state_num_id"]
            
            df= (
                df
                .reset_index()
                .rename(columns={"dias": "days"})
            )
            
            # add to DataFrame that collects all predictions
            df_predictions = pd.concat([df_predictions, df], ignore_index=True)

# show results
df_predictions

Beta E/0/1: 0.4030353619759821
Beta 2/3: 1.1435683754396773
Beta E/0/1: 0.29763696790638994
Beta 2/3: 1.456672208169223
Beta E/0/1: 0.38820909386296837
Beta 2/3: 1.1937605929001385
Beta E/0/1: 0.2921720556501724
Beta 2/3: 1.4710128417293546
Beta E/0/1: 0.3755576530881886
Beta 2/3: 1.234392387987179
Beta E/0/1: 0.2894220440216428
Beta 2/3: 1.4781831585094203
Beta E/0/1: 0.35678567333319716
Beta 2/3: 1.2917549222277065
Beta E/0/1: 0.27643795159559825
Beta 2/3: 1.5116446368163943
Beta E/0/1: 0.35112171272578363
Beta 2/3: 1.3084856613811933
Beta E/0/1: 0.28017264503515743
Beta 2/3: 1.5020842144429738
Beta E/0/1: 0.3387138406454973
Beta 2/3: 1.3443372452815237
Beta E/0/1: 0.27550121223657076
Beta 2/3: 1.5140347424097504
Beta E/0/1: 0.32249327738937966
Beta 2/3: 1.3897492515552738
Beta E/0/1: 0.26416650141344616
Beta 2/3: 1.5427160095300139
Beta E/0/1: 0.37768573677186795
Beta 2/3: 1.1699399386249612
Beta E/0/1: 0.2901070951274342
Beta 2/3: 1.4059692042989127
Beta E/0/1: 0.30574469601894194


Beta E/0/1: 0.4340746756646539
Beta 2/3: 1.2292762217206108
Beta E/0/1: 0.24668206070392076
Beta 2/3: 1.917644021624102
Beta E/0/1: 0.3079987687434487
Beta 2/3: 1.5921090878681066
Beta E/0/1: 0.2196696906187544
Beta 2/3: 1.8412599098995424
Beta E/0/1: 0.2856829324347378
Beta 2/3: 1.4707291877754882
Beta E/0/1: 0.2070718951197202
Beta 2/3: 1.6619676085386859
Beta E/0/1: 0.3521276363350439
Beta 2/3: 1.3747730213139833
Beta E/0/1: 0.23632433451203277
Beta 2/3: 1.6980169673901642
Beta E/0/1: 0.2562823096655236
Beta 2/3: 1.487671642416819
Beta E/0/1: 0.20386367858402793
Beta 2/3: 1.6079557874237371
Beta E/0/1: 0.2689101614733938
Beta 2/3: 1.5307654815632368
Beta E/0/1: 0.24001913372707775
Beta 2/3: 1.6024686493638969
Beta E/0/1: 0.30760037881867314
Beta 2/3: 1.4521689571881848
Beta E/0/1: 0.2162702989964616
Beta 2/3: 1.6853549019875662
Beta E/0/1: 0.2765108305975301
Beta 2/3: 1.5352988665520855
Beta E/0/1: 0.19084917428452347
Beta 2/3: 1.746703209990986
Beta E/0/1: 0.279726856497321
Beta 2/

Beta E/0/1: 0.3349792081672874
Beta 2/3: 1.5299773449171141
Beta E/0/1: 0.2405504245370966
Beta 2/3: 1.8099949733783665
Beta E/0/1: 0.41885516112529037
Beta 2/3: 1.3394354262443042
Beta E/0/1: 0.24483393228858763
Beta 2/3: 1.9753188774577013
Beta E/0/1: 0.41381783248663184
Beta 2/3: 1.3284478539100975
Beta E/0/1: 0.24179776336420208
Beta 2/3: 1.9322280851813793
Beta E/0/1: 0.2815859123814353
Beta 2/3: 1.6698012796843604
Beta E/0/1: 0.20959357869184553
Beta 2/3: 1.868050320870664
Beta E/0/1: 0.26220616826951215
Beta 2/3: 1.5297533917147477
Beta E/0/1: 0.19689105554775974
Beta 2/3: 1.6855772901143897
Beta E/0/1: 0.31660837124512736
Beta 2/3: 1.4808374411202299
Beta E/0/1: 0.22044001767230922
Beta 2/3: 1.7384224606496865
Beta E/0/1: 0.24272272058992025
Beta 2/3: 1.519444812796005
Beta E/0/1: 0.2018303021887292
Beta 2/3: 1.6124948117636202
Beta E/0/1: 0.23313484748415617
Beta 2/3: 1.6191993885173839
Beta E/0/1: 0.20914707434318328
Beta 2/3: 1.6765619227579107
Beta E/0/1: 0.2913272593440471

Beta E/0/1: 0.38912386710355845
Beta 2/3: 1.2919286114728838
Beta E/0/1: 0.2442985180831109
Beta 2/3: 1.7302096986178794
Beta E/0/1: 0.3332859244125789
Beta 2/3: 1.5354145804212167
Beta E/0/1: 0.2637561335231573
Beta 2/3: 1.744748147329142
Beta E/0/1: 0.4006124053877522
Beta 2/3: 1.4234200330083377
Beta E/0/1: 0.2399378596839242
Beta 2/3: 1.9903161286655642
Beta E/0/1: 0.3936187073168183
Beta 2/3: 1.4159522352537615
Beta E/0/1: 0.23095525779353812
Beta 2/3: 1.9643130250073904
Beta E/0/1: 0.26178893160681194
Beta 2/3: 1.7260611427237176
Beta E/0/1: 0.19839559741871565
Beta 2/3: 1.897519772938898
Beta E/0/1: 0.24387554591153576
Beta 2/3: 1.5746117867085836
Beta E/0/1: 0.18455062430753674
Beta 2/3: 1.7139089080052332
Beta E/0/1: 0.29605605272649593
Beta 2/3: 1.5389203376807934
Beta E/0/1: 0.2123993434680644
Beta 2/3: 1.7586252072794477
Beta E/0/1: 0.23782826094917353
Beta 2/3: 1.5307923736457139
Beta E/0/1: 0.20284746486724756
Beta 2/3: 1.6102252995936785
Beta E/0/1: 0.2151983535979928
Be

Beta E/0/1: 0.15792175262131813
Beta 2/3: 2.087992823375115
Beta E/0/1: 0.4033653347262757
Beta 2/3: 1.4535179153585318
Beta E/0/1: 0.2420114593011352
Beta 2/3: 2.044716739042145
Beta E/0/1: 0.3393410766906615
Beta 2/3: 2.07527008157025
Beta E/0/1: 0.20561816069121283
Beta 2/3: 2.6157641025859513
Beta E/0/1: 0.26201008323084285
Beta 2/3: 2.380813770460055
Beta E/0/1: 0.21473738269343715
Beta 2/3: 2.560702952974
Beta E/0/1: 0.3540972363934212
Beta 2/3: 2.01587984238976
Beta E/0/1: 0.22383295980230924
Beta 2/3: 2.5588173014717537
Beta E/0/1: 0.31326918774652585
Beta 2/3: 1.9782091282115017
Beta E/0/1: 0.22550235836581292
Beta 2/3: 2.294388515907743
Beta E/0/1: 0.36909625744990193
Beta 2/3: 2.0010454613632147
Beta E/0/1: 0.27447656643439994
Beta 2/3: 2.4316369434232086
Beta E/0/1: 0.40998189553983877
Beta 2/3: 1.4114038015568733
Beta E/0/1: 0.23410730944048755
Beta 2/3: 2.0528832222262317
Beta E/0/1: 0.2996953477119297
Beta 2/3: 1.928329138600448
Beta E/0/1: 0.2273248035841682
Beta 2/3: 2

...

Beta E/0/1: 0.12165038638167756
Beta 2/3: 2.384365410897479
Beta E/0/1: 0.10743733830391634
Beta 2/3: 2.424266044719018
Beta E/0/1: 0.11411521523405667
Beta 2/3: 2.521273303656184
Beta E/0/1: 0.0998221513598522
Beta 2/3: 2.5631660979264264
Beta E/0/1: 0.1214154225392633
Beta 2/3: 2.166652931270329
Beta E/0/1: 0.109417137098968
Beta 2/3: 2.197206325969586
Beta E/0/1: 0.1267921171865249
Beta 2/3: 2.115773629407849
Beta E/0/1: 0.11701960627415131
Beta 2/3: 2.140322879775924
Beta E/0/1: 0.12455901959080724
Beta 2/3: 2.0692097937160927
Beta E/0/1: 0.11368642703543824
Beta 2/3: 2.095787144990351
Beta E/0/1: 0.12559304488451098
Beta 2/3: 2.030223538032608
Beta E/0/1: 0.11473164476691136
Beta 2/3: 2.056311697981715
Beta E/0/1: 0.1267835025657605
Beta 2/3: 2.109042926103974
Beta E/0/1: 0.11701154675285542
Beta 2/3: 2.1335104858724327
Beta E/0/1: 0.13027624177974667
Beta 2/3: 2.308257854272263
Beta E/0/1: 0.11724010098205681
Beta 2/3: 2.344251257171135
Beta E/0/1: 0.13775600069748078
Beta 2/3: 2

Beta E/0/1: 0.09814064821578798
Beta 2/3: 1.957405170920179
Beta E/0/1: 0.09153146140754141
Beta 2/3: 1.9719792924701405
Beta E/0/1: 0.10033890604123868
Beta 2/3: 1.9532476834657735
Beta E/0/1: 0.09263557477320412
Beta 2/3: 1.9702572283401676
Beta E/0/1: 0.11098060878580608
Beta 2/3: 1.7324161279173482
Beta E/0/1: 0.10443850071922374
Beta 2/3: 1.7454360629259125
Beta E/0/1: 0.12351579866756797
Beta 2/3: 2.1034441851342263
Beta E/0/1: 0.11372430991637925
Beta 2/3: 2.127745480915251
Beta E/0/1: 0.12479008442656005
Beta 2/3: 2.2603134900260367
Beta E/0/1: 0.1139005001884924
Beta 2/3: 2.2894612445042397
Beta E/0/1: 0.11484879500874415
Beta 2/3: 2.153817003759803
Beta E/0/1: 0.1039026175982345
Beta 2/3: 2.1812007722148294
Beta E/0/1: 0.123606696495741
Beta 2/3: 2.1775840636540957
Beta E/0/1: 0.11271670644869501
Beta 2/3: 2.2055813262412776
Beta E/0/1: 0.13143351315020568
Beta 2/3: 2.3759118915769437
Beta E/0/1: 0.119490891089206
Beta 2/3: 2.409959991370241
Beta E/0/1: 0.12970154166004103
Be

Beta E/0/1: 0.11302029348284207
Beta 2/3: 2.5244958262923562
Beta E/0/1: 0.0987174486317832
Beta 2/3: 2.5663886205625976
Beta E/0/1: 0.12032847756337756
Beta 2/3: 2.1694305126066245
Beta E/0/1: 0.10832186227112071
Beta 2/3: 2.199983907305882
Beta E/0/1: 0.12462585105425099
Beta 2/3: 2.121229018378532
Beta E/0/1: 0.1148395206132022
Beta 2/3: 2.1457782687466076
Beta E/0/1: 0.12021913461634821
Beta 2/3: 2.0798407342257965
Beta E/0/1: 0.10931624033832915
Beta 2/3: 2.1064180855000547
Beta E/0/1: 0.12451034880831707
Beta 2/3: 2.032832354027519
Beta E/0/1: 0.113641336064552
Beta 2/3: 2.058920513976626
Beta E/0/1: 0.12136233108981044
Beta 2/3: 2.1226360148642285
Beta E/0/1: 0.11264860805120176
Beta 2/3: 2.144384956880637
Beta E/0/1: 0.12919421912592016
Beta 2/3: 2.3112573045138354
Beta E/0/1: 0.11614870700563869
Beta 2/3: 2.347250707412707
Beta E/0/1: 0.13560328386049508
Beta 2/3: 2.232659794461293
Beta E/0/1: 0.12479176632989625
Beta 2/3: 2.2618279215758474
Beta E/0/1: 0.10604885729751157
Bet

Beta E/0/1: 0.10909964471756427
Beta 2/3: 1.9338082036093227
Beta E/0/1: 0.10143652983285598
Beta 2/3: 1.9508177484837168
Beta E/0/1: 0.11098060878580608
Beta 2/3: 1.7324161279173482
Beta E/0/1: 0.10443850071922374
Beta 2/3: 1.7454360629259125
Beta E/0/1: 0.12459990981409283
Beta 2/3: 2.100744041158557
Beta E/0/1: 0.11481529211086919
Beta 2/3: 2.125045336939582
Beta E/0/1: 0.13344557003139396
Beta 2/3: 2.236995286443473
Beta E/0/1: 0.12370460655577119
Beta 2/3: 2.2632282654738565
Beta E/0/1: 0.12355141207199893
Beta 2/3: 2.1319099889957807
Beta E/0/1: 0.11266553743629908
Beta 2/3: 2.1592937574508078
Beta E/0/1: 0.1246914656804899
Beta 2/3: 2.1747843373953777
Beta E/0/1: 0.11380913280336107
Beta 2/3: 2.2027815999825586
Beta E/0/1: 0.13359457607080003
Beta 2/3: 2.369721327978162
Beta E/0/1: 0.12166934694868381
Beta 2/3: 2.4037694277714596
Beta E/0/1: 0.12752816272855386
Beta 2/3: 2.8954603092654034
Beta E/0/1: 0.11222510523802103
Beta 2/3: 2.948375807826382
Beta E/0/1: 0.1458783784190879

,days,S,E0,E1,I0,I1,I2,I3,R,D,N,E,scenario,model,date_prediction,state_num_id
0,1,4.565597e+07,1.000644e+05,6.670963e+04,2.755470e+04,6.140671e+04,2389.728066,497.860014,4.367000e+03,84.000000,4.591905e+07,1.667741e+05,worst,SEAPMDR,2020-03-28,35
1,2,1.396044e-09,3.278580e+07,1.010676e+07,8.520469e+05,1.985665e+06,6559.756994,534.861761,1.815116e+05,170.119344,4.591905e+07,4.289256e+07,worst,SEAPMDR,2020-03-28,35
2,3,-1.502009e-10,2.349205e+07,1.334298e+07,2.401212e+06,5.600759e+06,27414.526944,893.371300,1.053456e+06,283.465823,4.591905e+07,3.683503e+07,worst,SEAPMDR,2020-03-28,35
3,4,-6.815688e-14,1.683279e+07,1.326121e+07,3.895125e+06,9.086873e+06,66014.250543,2026.532316,2.774495e+06,517.519017,4.591905e+07,3.009400e+07,worst,SEAPMDR,2020-03-28,35
4,5,3.954639e-16,1.206122e+07,1.174656e+07,5.031299e+06,1.173822e+07,116438.288104,4248.530970,5.220026e+06,1033.420015,4.591905e+07,2.380779e+07,worst,SEAPMDR,2020-03-28,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2774039,87,2.211629e+06,NaN,NaN,NaN,2.270479e+03,864.849935,244.496192,7.801478e+05,10929.996403,3.007711e+06,1.624007e+03,best,SEIR,2021-01-19,53
2774040,88,2.211346e+06,NaN,NaN,NaN,2.217484e+03,844.731456,238.850212,7.805132e+05,10964.387711,3.007711e+06,1.585995e+03,best,SEIR,2021-01-19,53
2774041,89,2.211070e+06,NaN,NaN,NaN,2.165706e+03,825.071624,233.330554,7.808700e+05,10997.984553,3.007711e+06,1.548859e+03,best,SEIR,2021-01-19,53
2774042,90,2.210800e+06,NaN,NaN,NaN,2.115117e+03,805.860408,227.934625,7.812185e+05,11030.804718,3.007711e+06,1.512581e+03,best,SEIR,2021-01-19,53


In [13]:
df_predictions.to_csv("../data/br-states-simulacovid-predictions.csv", index=False)